In [14]:
import numpy as np
import pandas as pd

from sklearn.metrics import (
    confusion_matrix, 
    roc_curve, 
    roc_auc_score, 
    accuracy_score,
    precision_score, 
    recall_score, 
    f1_score
)
from sklearn.model_selection import (
    KFold, 
    train_test_split, 
    cross_val_score
)
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier


In [15]:
def evalua_metodo(modelo, X_train, y_train, X_test, y_test):
    """
    Evalúa el rendimiento de un modelo de clasificación.

    Parámetros:
    modelo: modelo de clasificación (debe implementar los métodos fit y predict_proba)
    X_train: conjunto de características de entrenamiento
    y_train: etiquetas de entrenamiento
    X_test: conjunto de características de prueba
    y_test: etiquetas de prueba

    Retorna:
    dict: un diccionario con las métricas evaluadas (matriz de confusión, AUC, accuracy)
    """
    
    # Ajustar el modelo con los datos de entrenamiento
    modelo.fit(X_train, y_train)
    
    # Realizar predicciones con los datos de prueba
    y_pred = modelo.predict(X_test)
    y_prob = modelo.predict_proba(X_test)[:, 1]
    
    # Calcular la matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    
    # Calcular la curva ROC y el AUC
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc = roc_auc_score(y_test, y_prob)
    
    # Calcular el accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    
    # Graficar la curva ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', label=f'AUC = {auc:.2f}')
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Curva ROC')
    plt.legend()
    plt.show()
    
    # Retornar las métricas
    metrics = {
        'matriz_confusion': matriz_confusion,
        'auc': auc,
        'accuracy': accuracy
    }
    
    return metrics


In [16]:
def cross_validation(modelo, X, y, k=5):
    """
    Realiza validación cruzada con k iteraciones.

    Parámetros:
    modelo: modelo de clasificación (debe implementar los métodos fit y predict_proba)
    X: conjunto de características
    y: etiquetas
    k: número de particiones para la validación cruzada, usamos 5 de valor predeterminado

    Retorna:
    list: una lista de diccionarios con las métricas evaluadas para cada partición
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    resultados = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        metricas = evalua_metodo(modelo, X_train, y_train, X_test, y_test)
        resultados.append(metricas)
    
    return resultados

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

def evalua_config(model, X, y, param_grid, scoring='neg_mean_squared_error', cv=5):
    """
    Evalúa distintas configuraciones de hiperparámetros utilizando validación cruzada.
    
    Parámetros:
    - model: el modelo de aprendizaje a evaluar (ej. una instancia de Ridge, Lasso, etc.)
    - X: matriz de características (features)
    - y: vector objetivo (target)
    - param_grid: lista de diccionarios, cada uno contiene una configuración de hiperparámetros
    - scoring: métrica de evaluación, por defecto 'neg_mean_squared_error' para MSE negativo
    - cv: número de folds para la validación cruzada, por defecto 5
    
    Retorna:
    - La configuración de hiperparámetros que genere el menor error promedio.
    """
    
    best_score = float('inf')
    best_params = None
    
    for params in param_grid:
        model.set_params(**params)
        scores = cross_val_score(model, X, y, scoring=scoring, cv=cv)
        mean_score = -np.mean(scores)  # Convertir a MSE positivo
        if mean_score < best_score:
            best_score = mean_score
            best_params = params
    
    return best_params, best_score

In [18]:
def evalua_config(model, X, y, param_grid, scoring='accuracy', cv=5):
    """
    Evalúa distintas configuraciones de hiperparámetros utilizando validación cruzada.
    
    Parámetros:
    - model: el modelo de aprendizaje a evaluar (ej. una instancia de Ridge, Lasso, etc.)
    - X: matriz de características (features)
    - y: vector objetivo (target)
    - param_grid: lista de diccionarios, cada uno contiene una configuración de hiperparámetros
    - scoring: métrica de evaluación, por defecto 'accuracy'
    - cv: número de folds para la validación cruzada, por defecto 5
    
    Retorna:
    - La configuración de hiperparámetros que genere el menor error promedio.
    """
    
    best_score = float('-inf')
    best_params = None
    
    for params in param_grid:
        model.set_params(**params)
        scores = cross_val_score(model, X, y, scoring=scoring, cv=cv)
        mean_score = np.mean(scores)
        if mean_score > best_score:
            best_score = mean_score
            best_params = params
    
    return best_params, best_score

def evalua_multiples_metodos(X, y):
    """
    Evalúa múltiples métodos de clasificación con diferentes configuraciones de hiperparámetros.
    
    Parámetros:
    - X: matriz de características (features)
    - y: vector objetivo (target)
    
    Retorna:
    - Un DataFrame con los resultados de evaluación para cada modelo y configuración.
    """
    
    resultados = []

    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Logistic Regression
    log_reg = LogisticRegression()
    param_grid_lr = [
        {'C': 0.01, 'penalty': 'l2'},
        {'C': 0.1, 'penalty': 'l2'},
        {'C': 1.0, 'penalty': 'l2'},
        {'C': 10.0, 'penalty': 'l2'}
    ]
    
    best_params_lr, _ = evalua_config(log_reg, X_train, y_train, param_grid_lr)
    log_reg.set_params(**best_params_lr)
    log_reg.fit(X_train, y_train)
    y_pred_lr = log_reg.predict(X_test)
    
    resultados.append({
        'Modelo': 'Regresión Logística',
        'Configuración': best_params_lr,
        'Accuracy': accuracy_score(y_test, y_pred_lr),
        'Precision': precision_score(y_test, y_pred_lr, average='macro'),
        'Recall': recall_score(y_test, y_pred_lr, average='macro'),
        'F1 Score': f1_score(y_test, y_pred_lr, average='macro')
    })
    
    # Linear Discriminant Analysis
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, y_train)
    y_pred_lda = lda.predict(X_test)
    
    resultados.append({
        'Modelo': 'Análisis de Discriminante Lineal',
        'Configuración': 'N/A',
        'Accuracy': accuracy_score(y_test, y_pred_lda),
        'Precision': precision_score(y_test, y_pred_lda, average='macro'),
        'Recall': recall_score(y_test, y_pred_lda, average='macro'),
        'F1 Score': f1_score(y_test, y_pred_lda, average='macro')
    })
    
    # K-Nearest Neighbors
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_test)
    
    resultados.append({
        'Modelo': 'K-Nearest Neighbors (K=3)',
        'Configuración': {'n_neighbors': 3},
        'Accuracy': accuracy_score(y_test, y_pred_knn),
        'Precision': precision_score(y_test, y_pred_knn, average='macro'),
        'Recall': recall_score(y_test, y_pred_knn, average='macro'),
        'F1 Score': f1_score(y_test, y_pred_knn, average='macro')
    })
    
    # Convertir resultados a DataFrame
    resultados_df = pd.DataFrame(resultados)
    return resultados_df
